In [1]:
#主要網址，但不是從這邊抓取
#url = 'https://aws.amazon.com/tw/ec2/pricing/reserved-instances/pricing'

# import package

In [2]:
import time
import datetime
# timestamp
t = time.time()
# datetime 物件
dt = datetime.datetime.fromtimestamp(t)

In [3]:
import sys, json
import urllib.request as ur
import requests

# 針對動態網頁的特性製作需要的資訊

In [4]:
timeSTP = '%d'%(time.mktime(dt.timetuple()))
timeSTP

'1606105678'

In [5]:
# 動態網頁抓取，製作選單
sysTYPE = ['linux','rhel','suse','windows','windows-std','windows-web','windows-enterprise','linux-std','linux-web','linux-enterprise']
len(sysTYPE)

10

In [6]:
#避免被server阻擋
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'}


In [7]:
#製作目標的url
#鎖定區域: ap-northeast-1
timeSTP = '%d'%(time.mktime(dt.timetuple()))
SYS = 0
url = 'https://a0.p.awsstatic.com/pricing/1.0/ec2/region/ap-northeast-1/previous-generation/reserved-instance/'+sysTYPE[SYS]+'/index.json?timestamp='+timeSTP


In [8]:
sysTYPE[SYS]

'linux'

# 開始爬網

In [9]:
#open url
result = ur.urlopen(url)

In [10]:
# another way to open url
#opener=ur.build_opener()
#opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]

#ur.install_opener(opener)
#result = ur.urlretrieve(url)


In [11]:
#result.read() #seem like a json file!

In [12]:
# json.loads將原本的text轉成json
resultJson = json.loads(result.read())

In [13]:
#檢視解析後的Json
resultJson

{'metadata': {'copyright': 'Copyright 2020 Amazon.com, Inc. or its affiliates. All rights reserved.',
  'disclaimer': 'This file is intended for use only on aws.amazon.com. We do not guarantee its availability or accuracy.',
  'format:version': '1.0',
  'source:url': 'https://plc-hawk-shared-prod-us-east-1.s3.amazonaws.com/offers/v1.0/aws/AmazonEC2/20201121194702/ap-northeast-1/index.json',
  'source:version': '20201121194702'},
 'prices': [{'id': 'PCB5ARVZ6TNS7A96.4NA7Y494T4.6YS6EN2CT7',
   'unit': 'Hrs',
   'price': {'USD': '0.5090000000'},
   'calculatedPrice': {'effectiveHourlyRate': {'USD': 0.509},
    'monthlyRate': {'USD': 371.57},
    'onDemandRate': {'USD': 0.77},
    'savingsOverOnDemandPercent': {'USD': 33.8961038961039},
    'upfrontRate': {'USD': 0}},
   'attributes': {'aws:ec2:capacitystatus': 'Used',
    'aws:ec2:clockSpeed': '2.5 GHz',
    'aws:ec2:currentGeneration': 'No',
    'aws:ec2:dedicatedEbsThroughput': '1000 Mbps',
    'aws:ec2:ecu': '26',
    'aws:ec2:enhanced

# 解析JSON檔並寫入csv檔

In [24]:
#先開一個檔案，寫入表頭
coluname = ['usagetype','operatingSystem','offerTermLeaseLength','offerTermOfferingClass','offerTermPurchaseOption',
          'instanceFamily','instanceType','region',
          'onDemandRate','upfrontRate','effectiveHourlyRate','monthlyRate','savingsOverOnDemandPercent']

fw = open(r'C:\Users\Fish\Downloads\AWS RI'+'/AWS_RI_'+sysTYPE[SYS]+'.csv','a+',encoding='ansi') 
fw.write(','.join([str(coluname[n]) for n in range(len(coluname))])+'\n')
fw.close()

In [25]:
len(resultJson['prices'])

270

In [26]:
#以下將抓取到的JSON檔轉輸出成.csv
for i in range(len(resultJson['prices'])): #2600 rows
    resultCalPri = resultJson['prices'][i]['calculatedPrice']
    resultATTR = resultJson['prices'][i]['attributes']
    lst = [
        resultATTR['aws:ec2:usagetype'], #和帳單上的一樣
        resultATTR['aws:ec2:operatingSystem'], #作業系統
        resultATTR['aws:offerTermLeaseLength'], #年期:一年、三年
        resultATTR['aws:offerTermOfferingClass'], #標準/可轉換
        resultATTR['aws:offerTermPurchaseOption'], #不預付/部分預付/完全預付
        resultATTR['aws:ec2:instanceFamily'], #一般用途
        resultATTR['aws:ec2:instanceType'], #型號
        resultATTR['aws:region'], #可用區域
        resultCalPri['onDemandRate']['USD'], #隨需價格
        resultCalPri['upfrontRate']['USD'], #預付金額
        resultCalPri['effectiveHourlyRate']['USD'], #按實際運行小時計費金額
        resultCalPri['monthlyRate']['USD'], #平均每月費用
        resultCalPri['savingsOverOnDemandPercent']['USD'],#透過隨需節省費用
    ]
    fw = open(r'C:\Users\Fish\Downloads\AWS RI'+'/AWS_RI_'+sysTYPE[SYS]+'.csv','a+',encoding='ansi') 
    fw.write(','.join([str(lst[n]) for n in range(len(lst))])+'\n')
    fw.close()
    

# done!
